In [2]:
# pip install yfinance

In [7]:
import yfinance as yf
from tqdm import tqdm
import pandas as pd

stocks = {
  "Technology": [
    {"name": "Microsoft Corporation", "ticker": "MSFT"},
    {"name": "Apple Inc.", "ticker": "AAPL"},
    {"name": "Alphabet Inc.", "ticker": "GOOGL"},
    {"name": "Intel Corporation", "ticker": "INTC"},
    {"name": "Cisco Systems, Inc.", "ticker": "CSCO"}
  ],
  "Consumer Discretionary": [
    {"name": "Amazon.com, Inc.", "ticker": "AMZN"},
    {"name": "Tesla, Inc.", "ticker": "TSLA"},
    {"name": "The Home Depot, Inc.", "ticker": "HD"},
    {"name": "McDonald's Corporation", "ticker": "MCD"},
    {"name": "Nike, Inc.", "ticker": "NKE"}
  ],
  "Healthcare": [
    {"name": "Johnson & Johnson", "ticker": "JNJ"},
    {"name": "Pfizer Inc.", "ticker": "PFE"},
    {"name": "Merck & Co., Inc.", "ticker": "MRK"},
    {"name": "Abbott Laboratories", "ticker": "ABT"},
    {"name": "Amgen Inc.", "ticker": "AMGN"}
  ],
  "Financials": [
    {"name": "JPMorgan Chase & Co.", "ticker": "JPM"},
    {"name": "Bank of America Corporation", "ticker": "BAC"},
    {"name": "Wells Fargo & Company", "ticker": "WFC"},
    {"name": "Goldman Sachs Group, Inc.", "ticker": "GS"},
    {"name": "Morgan Stanley", "ticker": "MS"}
  ],
  "Industrials": [
    {"name": "Boeing Company", "ticker": "BA"},
    {"name": "Caterpillar Inc.", "ticker": "CAT"},
    {"name": "General Electric Company", "ticker": "GE"},
    {"name": "3M Company", "ticker": "MMM"},
    {"name": "Honeywell International Inc.", "ticker": "HON"}
  ],
  "Consumer Staples": [
    {"name": "The Procter & Gamble Company", "ticker": "PG"},
    {"name": "Coca-Cola Company", "ticker": "KO"},
    {"name": "PepsiCo, Inc.", "ticker": "PEP"},
    {"name": "Walmart Inc.", "ticker": "WMT"},
    {"name": "Costco Wholesale Corporation", "ticker": "COST"}
  ]
}


In [31]:
stocks_list = [st["ticker"]  for key,val in stocks.items() for st in val]
counter = 1
for ticker in tqdm(stocks_list):
    if counter == 1:
        data = yf.download(ticker, start="2010-01-01", end="2024-10-03")
        data["stock"] = [ticker]*len(data)
    else:
        df = yf.download(ticker, start="2010-01-01", end="2024-10-03")
        df["stock"] = [ticker]*len(df)
        data._append(df, ignore_index=True)
        del df
    counter += 1
        

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


MSFT
AAPL
GOOGL
INTC
CSCO
AMZN


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


TSLA
HD
MCD
NKE
JNJ
PFE


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


MRK
ABT
AMGN
JPM
BAC
WFC
GS


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
 80%|████████  | 24/30 [00:00<00:00, 27.49it/s]

MS
BA
CAT
GE
MMM
HON


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
100%|██████████| 30/30 [00:01<00:00, 27.94it/s]

PG
KO
PEP
WMT
COST


In [33]:
data["stock"].unique()

array(['MSFT'], dtype=object)